# Get accuracy and consistency 
For GPT2-XL and Atlas-base for the relations P19 (_born in_), P101 (_specializes in_), P103 (_native language_).

In [4]:
import json
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import os
import math

from experiments.causal_trace import plot_trace_heatmap

## Load the data

In [55]:
relation = "P103"

### GPT2-XL

In [56]:
gpt2_file = f"/cephyr/users/lovhag/Alvis/projects/rome/data/{relation}_gpt2_xl_preds.jsonl"

In [57]:
gpt2_data = pd.DataFrame()
with open(gpt2_file) as f:
    for line in f:
        gpt2_data = gpt2_data.append(json.loads(line), ignore_index=True)
        
gpt2_data.head()

,attribute,candidate_p,candidate_prediction,candidate_rank,gold_p,gold_rank,known_id,prediction,prediction_p,prompt,relation_id,subject,template
0,French,0.376846,French,0.0,0.376846,0.0,0.0,French,0.376846,The native language of Louis Jules Trochu is,P103,Louis Jules Trochu,The native language of {} is
1,French,0.482079,French,0.0,0.482079,0.0,1.0,French,0.482079,The mother tongue of Louis Jules Trochu is,P103,Louis Jules Trochu,The mother tongue of {} is
2,French,0.636831,Italian,0.0,0.027837,1.0,2.0,Italian,0.636831,The native language of Valeria Bruni Tedeschi is,P103,Valeria Bruni Tedeschi,The native language of {} is
3,French,0.717344,Italian,0.0,0.022046,1.0,3.0,Italian,0.717344,The mother tongue of Valeria Bruni Tedeschi is,P103,Valeria Bruni Tedeschi,The mother tongue of {} is
4,French,0.316919,French,0.0,0.316919,0.0,4.0,French,0.316919,The native language of Pierre Messmer is,P103,Pierre Messmer,The native language of {} is


### Atlas-base
Only read the results corresponding to GPT2-XL (same template).

Comment: We could also load results with confidences given, but these are only across the options and not comparable to the confidences of GPT2-XL.

In [12]:
def template_ends_with_mask(template):
    return template.replace(".","").strip()[-3:]=="[Y]"

In [58]:
#atlas_base_file = "/cephyr/users/lovhag/Alvis/projects/atlas/data/experiments/pararel-eval-zero-shot-base-no-space-likelihood-no-eos-with-3/P19-base-2017-1115963/P19-step-0.jsonl"
#atlas_base_file = "/cephyr/users/lovhag/Alvis/projects/atlas/data/experiments/pararel-eval-zero-shot-base-no-space-likelihood-no-eos-with-3/P101-base-2017-1115964/P101-step-0.jsonl"
atlas_base_file = "/cephyr/users/lovhag/Alvis/projects/atlas/data/experiments/pararel-eval-zero-shot-base-no-space-likelihood-no-eos-with-3/P103-base-2017-1115947/P103-step-0.jsonl"

assert relation in atlas_base_file

In [59]:
atlas_base_data = pd.DataFrame()
with open(atlas_base_file) as f:
    for line in f:
        tmp_data = json.loads(line)
        if template_ends_with_mask(tmp_data["pattern"]):
            del tmp_data["passages"]
            del tmp_data["metadata"]
            atlas_base_data = atlas_base_data.append(tmp_data, ignore_index=True)

atlas_base_data["answers"] = atlas_base_data.answers.apply(lambda val: val[0])
atlas_base_data["pattern"] = atlas_base_data.pattern.apply(lambda val: val.replace("[X]", "{}").replace(" [Y].","").replace(" [Y]",""))
atlas_base_data["query"] = atlas_base_data["query"].apply(lambda val: val.replace("<extra_id_0>.","").replace("<extra_id_0>",""))
            
atlas_base_data = atlas_base_data.rename(columns = {"answers": "attribute", 
                                         "generation": "prediction",
                                         "generation_by_choice": "candidate_prediction",
                                         "pattern": "template",
                                         "query": "prompt",
                                         "sub_label": "subject"
                                         }
                                        )
atlas_base_data.head()

,attribute,prediction,candidate_prediction,template,prompt,subject
0,French,French French French. The French language -Jules,French,The native language of {} is,The native language of Louis Jules Trochu is,Louis Jules Trochu
1,French,French Italian French. She is Italian -French ...,French,The native language of {} is,The native language of Valeria Bruni Tedeschi is,Valeria Bruni Tedeschi
2,French,French French French. The French language Jacq...,French,The native language of {} is,The native language of Pierre Messmer is,Pierre Messmer
3,French,French French French d'Anjou. he was born Fren...,French,The native language of {} is,The native language of Roger Nimier is,Roger Nimier
4,Russian,Russian Russian Russian. The Russian language ...,Russian,The native language of {} is,The native language of Andrey Malakhov is,Andrey Malakhov


Can confirm that all data query values are the same!

In [60]:
gpt2_data_to_comp = gpt2_data.drop(columns=["candidate_p", "candidate_rank", "gold_p", "gold_rank", "known_id", "prediction_p", "relation_id"]
              ).sort_index(axis=1).sort_values(by=["subject", "template"], ignore_index=True)
atlas_base_data_to_comp = atlas_base_data.sort_index(axis=1).sort_values(by=["subject", "template"], ignore_index=True)


gpt2_data_to_comp.compare(atlas_base_data_to_comp, align_axis="columns")

candidate_prediction          prediction  \
                     self    other       self   
0                   Hindi    Hindi        Mar   
1                   Hindi  English        Mar   
2                   Dutch    Dutch        the   
3                   Dutch    Dutch        the   
4                 Italian  Italian    Italian   
...                   ...      ...        ...   
1829               German  Swedish        the   
1830              Russian  Russian    Russian   
1831              Russian  Russian    Russian   
1832              Russian  Russian      Uzbek   
1833              Russian  Russian      Uzbek   

                                                         
                                                  other  
0     Hindi Hindi Hindi a former President of India,...  
1         Bengali English Bengali he is the President.   
2     Dutch Dutch Dutch a Dutch writer of poetry was...  
3        Dutch Dutch Dutch a Dutch writer, Aagje Deken   
4     Italian Italian Italian a racecar driver, Varz...  
...                                                 ...  
1829  Swedish Swedish Swedish, although it is not sp...  
1830  Russian Russian Russian. The Russian language ...  
1831  Russian Russian Russian. The Russian language ...  
1832                  Chechen Chechen Chechen  елим ан   
1833                  Chechen Chechen Chechen  елим ан   

[1834 rows x 4 columns]

## Get accuracy and consistency

In [61]:
# accuracy across all paraphrases

def get_accuracy(data, pred_type):
    return (data.attribute==data[pred_type].apply(lambda val: val.strip())).sum()/len(data)    

def get_consistency(data, pred_type):
    def get_consistency_per_attribute(vals):
        num_cons = 0
        for i in range(len(vals)-1):
            num_cons += sum(vals[i]==vals[i+1:])
        return num_cons, math.comb(len(vals), 2)
    
    num_cons = 0
    num_tot = 0
    for subject in data.subject.unique():
        tmp_num_cons, tmp_num_tot = get_consistency_per_attribute(data[data.subject==subject][pred_type].reset_index(drop=True))
        num_cons += tmp_num_cons
        num_tot += tmp_num_tot
        
    return num_cons/num_tot

In [62]:
gpt2_data.groupby(by="template").apply(lambda val: get_accuracy(val, "prediction"))

template
The mother tongue of {} is      0.724100
The native language of {} is    0.728462
dtype: float64

In [63]:
atlas_base_data.groupby(by="template").apply(lambda val: get_accuracy(val, "prediction"))

template
The mother tongue of {} is      0.0
The native language of {} is    0.0
dtype: float64

In [64]:
gpt2_data.groupby(by="template").apply(lambda val: get_accuracy(val, "candidate_prediction"))

template
The mother tongue of {} is      0.763359
The native language of {} is    0.765540
dtype: float64

In [65]:
atlas_base_data.groupby(by="template").apply(lambda val: get_accuracy(val, "candidate_prediction"))

template
The mother tongue of {} is      0.931298
The native language of {} is    0.947655
dtype: float64

In [66]:
print(f"GPT2 candidate consistency: {get_consistency(gpt2_data, 'candidate_prediction')}")
print(f"Atlas-base candidate consistency: {get_consistency(atlas_base_data, 'candidate_prediction')}")

GPT2 candidate consistency: 0.95092693565976
Atlas-base candidate consistency: 0.960741548527808
